In [1]:
from google.colab import drive
import shutil
import torch
from torchvision import transforms
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

Dataset

In [2]:
drive.mount ('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
## Hyper Parameters
batch_size = 4

Loading the dataset

In [4]:
# Transformer to tensor
img_size = 256

transformer=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
])  

In [5]:
dataset_path = '/content/gdrive/MyDrive/Train-Test-Val/'

In [6]:
def load_dataset(d_path):
    train_dataset_manual = torchvision.datasets.ImageFolder(d_path, transform=transformer)
    train_loader_manual = torch.utils.data.DataLoader(train_dataset_manual)
    return train_loader_manual

In [7]:
train_dataset = load_dataset(str(dataset_path + 'train')).dataset
test_dataset = load_dataset(str(dataset_path + 'test')).dataset
valid_dataset = load_dataset(str(dataset_path + 'val')).dataset

In [8]:
train_dataset = torch.utils.data.ConcatDataset ([train_dataset, test_dataset])

In [9]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=2, shuffle=True)

In [10]:
train_count = (len(train_dataset))
test_count = len(test_dataset)

In [11]:
print('Train Set- ' + str(len(train_dataset)) + ' images in ' + str(len(train_loader)) +' batches')
#print('Testing Set - ' + str(len(test_dataset)) + ' images in ' + str(len(test_loader)) + ' batches' )
print('Validation Set - ' + str(len(valid_dataset)) + ' images in ' + str(len(valid_loader)) + ' batches')

Train Set- 5565 images in 1392 batches
Validation Set - 1388 images in 347 batches


Network

In [12]:
class Transformer_Branch (nn.Module):
    def __init__(self):
        super().__init__()

        self.SWIN = torchvision.models.swin_b()
        self.SWIN.head = nn.Linear(in_features=1024, out_features=7, bias=True)

    def forward (self, x):

      x = self.SWIN (x)

      return x

In [13]:
class Overall_Arch (nn.Module):
    def __init__(self):
        super().__init__()

        self.Transformer_Branch = Transformer_Branch()
    
    def forward (self, x):

      f = self.Transformer_Branch(x)

      return f

Model

In [14]:
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [16]:
model = Overall_Arch().to(device)

In [17]:
#Load saved model
#save_path = '/content/gdrive/MyDrive/Train-Test-Val/Epochn35'
#model = torch.load(save_path)

In [18]:
model.eval()

Overall_Arch(
  (Transformer_Branch): Transformer_Branch(
    (SWIN): SwinTransformer(
      (features): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (1): Sequential(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attn): ShiftedWindowAttention(
              (qkv): Linear(in_features=128, out_features=384, bias=True)
              (proj): Linear(in_features=128, out_features=128, bias=True)
            )
            (stochastic_depth): StochasticDepth(p=0.0, mode=row)
            (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (mlp): MLP(
              (0): Linear(in_features=128, out_features=512, bias=True)
              (1): GELU(approximate='none')
              (2): Dropout(p=0.0, inplace=False)
              (3

Running

In [19]:
num_epochs = 1
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.0001

In [20]:
optimizer=optim.Adam(model.parameters(),lr=LEARNING_RATE,weight_decay=WEIGHT_DECAY)
loss_function=nn.CrossEntropyLoss()

In [21]:
for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()

    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss += loss.item()
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on validation dataset
    model.eval()
    
    valid_accuracy = 0.0
    valid_loss = 0.0

    for i, (images,labels) in enumerate(valid_loader):
        if torch.cuda.is_available():
          images=Variable(images.cuda())
          labels=Variable(labels.cuda())
            
        outputs=model(images)

        loss = loss_function (outputs, labels)
        _,prediction=torch.max(outputs.data,1)
        valid_accuracy += int(torch.sum(prediction == labels.data))
        valid_loss+= loss.item()
        
    valid_accuracy = valid_accuracy/test_count
    valid_loss = valid_loss/test_count
        
    print('Epoch: %d Train Accuracy: %.5f Train Loss: %.5f Validation Accuracy: %.5f  Validation Loss: %.5f' % (epoch, train_accuracy, train_loss, valid_accuracy, valid_loss))

KeyboardInterrupt: ignored

In [28]:
#Save Model
#save_path = '/content/gdrive/MyDrive/Train-Test-Val/Epochn45'
#torch.save(model, save_path)

Testing the Model

In [26]:
#path for testing dataset
dataset_path = '/content/gdrive/MyDrive/Train_Test_Folder/'

In [27]:
test_dataset = load_dataset(str(dataset_path + 'test')).dataset
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
print('Testing Set - ' + str(len(test_dataset)) + ' images in ' + str(len(test_loader)) + ' batches' )

Testing Set - 254 images in 64 batches


In [34]:
#Load latest model
#save_path = '/content/gdrive/MyDrive/Train-Test-Val/Epochn35'
#model = torch.load(save_path)

In [ ]:
model.eval()

In [30]:
from sklearn.metrics import classification_report, accuracy_score
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:
y_pred = []
y_true = []

# iterate over test data
for i, (images, labels) in enumerate(test_loader):
    images, labels = images.to(device), labels.to(device)

    output = model(images) # Feed Network

    output = (torch.max(torch.exp(output), 1)[1])
    output = output.data.cpu().numpy()

    y_pred.extend(output) # Save Prediction
        
    labels1 = labels.data.cpu().numpy()
    y_true.extend(labels1) # Save Truth

In [32]:
r = classification_report(y_true, y_pred,zero_division=0,output_dict=True)

In [33]:
print ('Accuracy - ', r.get ('accuracy'))
print ('Weighted Average - ', r.get('weighted avg'))
print ('Macro Average - ', r.get('macro avg'))

Accuracy -  0.2992125984251969
Weighted Average -  {'precision': 0.08952817905635813, 'recall': 0.2992125984251969, 'f1-score': 0.13781913624433312, 'support': 254}
Macro Average -  {'precision': 0.04274465691788527, 'recall': 0.14285714285714285, 'f1-score': 0.06580086580086582, 'support': 254}
